In [ ]:
import os
import re
import sys
import glob
import json
import shutil
import skimage
import tifffile
import datetime
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import ndimage
from matplotlib import pyplot as plt
from matplotlib import colors as mplcolors

from skimage import feature
from skimage import morphology

%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.append('/Users/keith.cheveralls/projects/opencell-process/')
from pipeline_process.imaging import image, utils, viz

sys.path.append('/Users/keith.cheveralls/projects/dragonfly-automation/')
import dragonfly_automation.utils
from dragonfly_automation.fov_models import PipelineFOVScorer
from dragonfly_automation.qc.pipeline_plate_qc import PipelinePlateQC

In [ ]:
ml_dirs = glob.glob('/Users/keith.cheveralls/image-data/dragonfly-automation-tests/ML*')
ml_dirs = sorted(ml_dirs)
ml_dirs

In [ ]:
for ml_dir in ml_dirs:
    qc = PipelinePlateQC(ml_dir)
    qc.summarize()

In [ ]:
qc = PipelinePlateQC('/Users/keith.cheveralls/image-data/dragonfly-automation-tests/ML0212_20191112/')

In [ ]:
qc.summarize()

In [ ]:
qc.plot_counts_and_scores(save_plot=True)

In [ ]:
qc.generate_z_projections('/Users/keith.cheveralls/projects/opencell-process/')

In [ ]:
qc.tile_fovs(channel_ind=1, save_plot=True)